In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import sklearn.dummy
import sklearn.metrics
import sklearn.model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


In [ ]:
#Lecture du Fichier
full_df = pd.read_csv('/kaggle/input/retardavionclean/retardAvionClean.csv', index_col=None, sep = '\t', encoding='UTF-8')
full_df.drop({'Unnamed: 0','DEST_CITY_NAME','ORIGIN_CITY_NAME'}, axis=1, inplace=True)


# Création d'un échantillon avec 10% des données

In [ ]:
ind = np.random.choice(full_df.shape[0], round(full_df.shape[0]*0.1))
sub_df = full_df.iloc[ind]
#ind = np.random.choice(df_for_reg_lineaire.shape[0], round(df_for_reg_lineaire.shape[0]*0.1))
#suf_df_for_reg_lineaire = df_for_reg_lineaire.iloc[ind]

# Création du jeu d'entrainement et de test
* Utlisation de train_test_split qui va découper les données avec le pourcentage spécifié

In [ ]:
from sklearn.model_selection import train_test_split
# On récupère les features d'un côté...
X = sub_df.drop('ARR_DELAY', axis=1).to_numpy()
#X_forReg = suf_df_for_reg_lineaire.drop('ARR_DELAY', axis=1).to_numpy()

# et les labels de l'autre
y = sub_df[['ARR_DELAY']].to_numpy()
#y_forReg = suf_df_for_reg_lineaire[['ARR_DELAY']].to_numpy()

#Création d'un jeu d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#X_train_forReg, X_test_forReg, y_train_forReg, y_test_forReg = train_test_split(X_forReg, y_forReg, test_size=0.33)

#Suppression des variables temporaires
del X,y

# Standardisation
Normalisation et réduction du jeu d'entrainement et du jeu de test

In [ ]:
#Création des jeux de données standardisés
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)

#std_scale = preprocessing.StandardScaler().fit(X_test)
X_test_std = std_scale.transform(X_test)

#Pour la régression linéaire
#std_scale = preprocessing.StandardScaler().fit(X_train_forReg)
#X_train_forReg_std = std_scale.transform(X_train_forReg)

#std_scale = preprocessing.StandardScaler().fit(X_test_forReg)
#X_test_forReg_std = std_scale.transform(X_test_forReg)

# Création de différents modèles
## Baseline
Création d'un modèle très basique pour avoir une référence basse

* Utlisation du modèle DummyRegressort qui ressort la moyenne

In [ ]:
model = sklearn.dummy.DummyRegressor()
model.fit(X_train_std, y_train)
preds = model.predict(X_test_std)
print(sklearn.metrics.r2_score(y_test, preds))
print(sklearn.metrics.mean_absolute_error(y_test, preds))

In [ ]:
CROSS VALIDATION 5FOLDS
model = sklearn.dummy.DummyRegressor()
sklearn.model_selection.cross_val_score(model, X_train_std, y_train, cv=tscv.split(X_train_std, y_train))

# Indicateurs Regression
* explained_variance = metrics.explained_variance_score
* max_error = metrics.max_error
* neg_mean_absolute_error = metrics.mean_absolute_error
* neg_mean_squared_error = metrics.mean_squared_error
* neg_root_mean_squared_error = metrics.mean_squared_error
* neg_mean_squared_log_error = metrics.mean_squared_log_error
* neg_median_absolute_error = metrics.median_absolute_error
* R2 = metrics.r2_score
* neg_mean_poisson_deviance = metrics.mean_poisson_deviance
* neg_mean_gamma_deviance = metrics.mean_gamma_deviance
* neg_mean_absolute_percentage_error = metrics.mean_absolute_percentage_error

In [ ]:
#Fonction qui consolide les différents modèles dans un tableau avec leurs résultats
def consolidationResultat_model(resultat, model, X_test, y_reel , nomModel):
    y_pred = model.predict(X_test)
    r2 = sklearn.metrics.r2_score(y_reel, y_pred)
    mae = sklearn.metrics.mean_absolute_error(y_reel, y_pred)
    SqrError = sklearn.metrics.mean_squared_error(y_reel, y_pred)
    resultat[nomModel] = {r2,mae,SqrError}
    return resultat

In [ ]:
#GRID SEARCH pour trouver les HYPERPARAMETRES ==> PAS DANS CET EXEMPLE
params = {}
model = sklearn.dummy.DummyRegressor()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test,'DummyRegressor')

In [ ]:
#GRID SEARCH sur régression linéaire ==> PAS DE PARAMETRE DANS CET EXEMPLE
params = {}
model = sklearn.linear_model.LinearRegression()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'Régression Linéaire')

#Vérifier si une obeservation est complètement aberrante

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit(X_train_std, y_train)
y_pred  = model.predict(X_test_std)

print(sklearn.metrics.mean_absolute_error(y_test, y_pred))
print(sklearn.metrics.r2_score(y_test, y_pred))

In [ ]:
fig = plt.figure(1, figsize=(6, 8))
plt.barh(full_df.loc[:, full_df.columns != 'ARR_DELAY'].columns.values,model.coef_[0],)

In [ ]:
plt.barh(full_df.loc[:, full_df.columns != 'ARR_DELAY'].columns.values,model.feature_importances_)

In [ ]:
fig = plt.figure()
fig.suptitle('Prediction/Réel')
plt.xlabel('Valeur Réelle')
plt.ylabel('Prédiction')
plt.scatter(y_test, y_pred,color = 'coral', alpha=0.1)

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(y_train_forReg)
plt.ylim(-100,250)
plt.show()

In [ ]:
#GRID SEARCH avec trouver l'Alpha de la régularisation Ridge
params = {'alpha':[1, 10]}
model = sklearn.linear_model.Ridge()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'Régression Ridge')

# Avantages de l’arbre de régression par rapport au modèle de régression linéaire sont les suivants :
* Il est plus simple et plus direct dans son approche
* La structure liant Y à X1, . . . , Xp n’importe pas ; celle-ci peut être linéaire ou autre
* Il n’y a pas d’hypothèse mathématique sous-jacente (pas d’hypothèse de normalité ou autre)
* Les dépendances éventuelles entre X1, . . . , Xp ne posent pas de problème

In [ ]:
#GRID SEARCH sur arbre de décision
params = {'max_depth': range(8, 9, 1)}
model = sklearn.tree.DecisionTreeRegressor()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'Arbre de décision')

#MAX_DEPTH à 8 semble être le meilleur hyperparamètre
#Utiliser les variables les plus importantes features importance

In [ ]:
model = sklearn.tree.DecisionTreeRegressor(max_depth = 8)
model.fit(X_train_std, y_train)

y_pred  = model.predict(X_test_std)

fig = plt.figure()
fig.suptitle('Prediction/Réel')
plt.xlabel('Valeur Réelle')
plt.ylabel('Prédiction')
plt.scatter(y_test, y_pred,color = 'coral', alpha=0.1)

In [ ]:
#GRID SEARCH sur Random Forest
from sklearn.ensemble import RandomForestRegressor
params = {'max_depth': range(8, 14, 2)}
model = sklearn.ensemble.RandomForestRegressor(n_estimators = 30)
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'RandomForest')

#max_depth=10 pour la valeur optimale. Refaire avec n_estimators plus grand

In [ ]:
#GRID SEARCH sur LGBM
import lightgbm as lgb
params = {
    'max_depth': range(13, 16,2)
}
model = lgb.LGBMRegressor()
clf = GridSearchCV(model, params,  cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test,'LGBM')
#max_depth=15 pour la valeur optimale.

In [ ]:
#GRID SEARCH KNN Regressor
params = {
    'n_neighbors': range(5, 12,1)
}
model = sklearn.neighbors.KNeighborsRegressor()
clf = GridSearchCV(model, params,  cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test,'KNN Regressor')

#n_neighbors=10 valeur optimale

In [ ]:
model = sklearn.neighbors.KNeighborsRegressor(n_neighbors = 10)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
r2 = sklearn.metrics.r2_score(y_test, y_pred)

# Sérialisation du modèle pour le site Web

In [ ]:
#Serialisation du modèle de regression linéaire et du Scaler
pickle.dump(model, open('RegressionLineaire.pickle', 'wb')) #Saving the model
pickle.dump(std_scale, open('scaler.pickle', 'wb')) #Saving the scaler

In [ ]:
#Serialisation du modèle d'arbre de décision
pickle.dump(model, open('arbreDecision.pickle', 'wb')) #Saving the model